In [1]:
import pennylane as qml
import numpy as np

In [ ]:
## code for VGQEC 3 encoding
def VGQEC_3_encoding(alpha, wires):
    A = (1.0 + np.exp(1j * alpha)) / 2.0
    B = (1.0 - np.exp(1j * alpha)) / 2.0
    ancillas = np.array([A, 0, 0, B], dtype=complex)
    qml.MottonenStatePreparation(ancillas, wires=[1, 2])

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])


In [ ]:
#testing the encoding circuit
dev = qml.device("default.qubit", wires=3)
@qml.qnode(dev)
def VGQEC_3(alpha, wires):
    qml.Hadamard(wires=0) # Initial state 
    VGQEC_3_encoding(alpha, wires)

    return qml.state()

print(VGQEC_3(np.pi/2, dev.wires))

[0.35355339+0.35355339j 0.        +0.j         0.        +0.j
 0.35355339-0.35355339j 0.35355339-0.35355339j 0.        +0.j
 0.        +0.j         0.35355339+0.35355339j]


In [10]:
#variational recovery circuit for 3 qubits
def recovery_variational_3(beta, L = 2, wires=3):
    for i in range(wires):
        qml.RZ(beta[1], wires=i)

    for l in range(L):
        for i in range(wires):
            qml.RX(beta[0], wires=i)
            qml.RZ(beta[1], wires=i)        
    
        qml.CZ(beta[2], wires=[0, 1])
        qml.CZ(beta[2], wires=[1, 2])
        qml.CZ(beta[2], wires=[0, 2])
        

    for i in range(wires):
        qml.RX(beta[0], wires=i)
        qml.RZ(beta[1], wires=i)

In [9]:
#reovery map circuit for 3 qubits (from the repetition code)
def repetition_code_recovery_3(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 1])
    qml.Toffoli(wires=[2, 1, 0])